In [ ]:
import os
import numpy as np
import h5py
import tensorflow as tf
from flask import Flask, request, jsonify, render_template
from PIL import Image
import io
import glob
from utils import recall_m, precision_m, f1_m  # Custom metrics, if any

# Initialize Flask app
app = Flask(__name__)

# Load the trained model
model = tf.keras.models.load_model("model_save.h5", custom_objects={"f1_m": f1_m, "precision_m": precision_m, "recall_m": recall_m})

# Configure upload folder
UPLOAD_FOLDER = 'static/uploads/'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024  # Max file size: 16MB

# Preprocess the uploaded image to match the model's input format
def preprocess_image(image):
    # Convert the image to numpy array and preprocess (same as done in your training)
    data = np.array(image)
    
    # Example preprocessing steps, similar to what was in your training
    mid_rgb = data[:, :, 1:4].max() / 2.0
    mid_slope = data[:, :, 12].max() / 2.0
    mid_elevation = data[:, :, 13].max() / 2.0
    
    data_red = data[:, :, 3]
    data_nir = data[:, :, 7]
    data_ndvi = np.divide(data_nir - data_red, np.add(data_nir, data_red))
    
    VAL_XX = np.zeros((1, 128, 128, 6))  # Single image with shape (128, 128, 6)
    VAL_XX[0, :, :, 0] = 1 - data[:, :, 3] / mid_rgb  # RED
    VAL_XX[0, :, :, 1] = 1 - data[:, :, 2] / mid_rgb  # GREEN
    VAL_XX[0, :, :, 2] = 1 - data[:, :, 1] / mid_rgb  # BLUE
    VAL_XX[0, :, :, 3] = data_ndvi  # NDVI
    VAL_XX[0, :, :, 4] = 1 - data[:, :, 12] / mid_slope  # SLOPE
    VAL_XX[0, :, :, 5] = 1 - data[:, :, 13] / mid_elevation  # ELEVATION

    return VAL_XX

# Route for homepage
@app.route('/')
def index():
    return render_template('index.html')

# Route for image prediction
@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({"error": "No file part"})
    
    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No selected file"})
    
    if file:
        # Save the uploaded file temporarily
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
        file.save(file_path)
        
        # Open the image (assuming it's in .h5 format, adjust this if it's different)
        with h5py.File(file_path, 'r') as hdf:
            data = np.array(hdf.get('img'))
        
        # Preprocess the image for the model
        VAL_XX = preprocess_image(data)
        
        # Make prediction
        pred_img = model.predict(VAL_XX)
        pred_img = (pred_img > 0.5).astype(np.uint8)  # Apply threshold
        
        # For now, let's return the prediction result as a JSON response (you can customize this)
        return jsonify({"prediction": pred_img.tolist()})

if __name__ == '__main__':
    # Run the Flask app
    app.run(debug=True)
